# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): Щекотов Иван

### Общая информация

__Дата выдачи:__ 08.04.2020

__Дедлайн:__ 23:59 22.04.2020


### Оценивание и штрафы

Оценка за ДЗ вычисляется по следующей формуле:

$$
\min(\text{points}, 21)  \times 10 / 21,
$$

где points — количество баллов за домашнее задание, которое вы набрали. Максимальное число баллов, которое можно получить за решение данного домашнего задания — 24, все баллы сверх 21 идут в бонус (таким образом, за данное домашнее задание можно получить 3 бонусных балла). Накопленные бонусные баллы можно будет потом распределять по другим домашним заданиям и проверочным (+1 бонусный балл = +1 к оценке за домашнее задание/проверочную).

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи

Загрузка файлов с решениями происходит в системе [Anytask](https://anytask.org/).

Инвайт для группы ИАД-4: zG1cIyT

Перед отправкой перезагрузите ноутбук и проверьте, что все ячейки могут быть последовательно выполнены. Ноутбук должен запускаться с использованием python 3.6+

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [2]:
# чтобы видеть проход по итерациям, можно использовать библиотеку tqdm
# она работает примерно так:
from tqdm import tqdm
import time

for i in tqdm(range(100)):
    time.sleep(0.1)

100%|██████████| 100/100 [00:10<00:00,  9.56it/s]


### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - `'title'`
 - `'description'`
 - `'Category_name'`
 - `'Category'`

Имеется информация об объектах 50 классов.
Задача: по новым объектам (`'title'`, `'description'`) предсказать `'Category'`.
(Очевидно, что параметр `'Category_name'` для предсказания классов использовать нельзя)

In [3]:
data = pd.read_csv("avito_data.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [4]:
data.shape

(30000, 4)

In [5]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

In [8]:
y_train[:5]

array([ 27,  20,  84, 106,  27])

In [9]:
X_test

array([['Винт сдвижной панели потолка бмв Х5 Е53',
        '"*3308* Винт сдвижной панели потолка oem: 54107199476 (54 10 7 199 476) /\n/\n Запчасть Б/У в хорошем состоянии для BMW X5 E53, /\nдемонтирована с автомобиля из Японии без пробега по РФ. Гарантия на все запчасти 14 дней. Возможна замена/установка в автосервисе партнеров. /\nПримечания: / /\nДля упрощения поиска запчасти сообщите нам внутренний код: *3308* /\nВ наличии на складе: 16 шт. /\nПроверяйте соответствие данной запчасти по оригинальному номеру ОЕМ - Вашему автомобилю по VIN. Более 1000 наименований запасных частей для BMW на нашем складе в Москве. Отправляем по всей России почтой или транспортными компаниями"'],
       ['Сандалики Totto для мальчика',
        'Продаю сандалики Totto для малыша! Размер 20,по стельке 12 см.'],
       ['Фара правая Toyota RAV 4 галоген 2015-19',
        'Фара  правая  для  Toyota  RAV4   2015/\nОригинальный номер: 8113042650/\nтойота рав4 тоета рав 4/\nПроизводитель: Toyota/\nСостояние: о

In [10]:
def compression(array_to_compress: np.array):
    '''
    Function to compress sparse matrices for model fitting
    '''
    print('{} array in memory (raw): {:.3f} Mb'.format(str(array_to_compress), array_to_compress.nbytes * 1e-6))
    csr = csr_matrix(array_to_compress)
    print('{} array in memory (compressed): {:.3f} Mb'.format(
        str(array_to_compress), (csr.data.nbytes + csr.indptr.nbytes + csr.indices.nbytes) * 1e-6
    ))
    return csr

### Токенизация (0.5 балла)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации. В данном задании мы будем использовать `WordPunctTokenizer`.

In [11]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'

print("before:", text,)
print("after:", tokenizer.tokenize(text.lower()))

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: ['здраствуйте', '.', 'я', ',', 'кирилл', '.', 'хотел', 'бы', 'чтобы', 'вы', 'сделали', 'игру', ',', '3д', '-', 'экшон', 'суть', 'такова', '...']


__Задание:__ реализуйте функцию ниже.

In [12]:
def preprocess(text: str, tokenizer) -> str:
    """
    Данная функция принимает на вход текст, 
    а возвращает тот же текст, но с пробелами между каждым токеном
    """
    
    return ' '.join(tokenizer.tokenize(text.lower()))

In [13]:
assert preprocess(text, tokenizer) == 'здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...'

__Задание:__ токенизируйте `'title'` и `'description'` в `train` и `test`.

In [14]:
def tokenize_array(a: np.array, tokenizer):
    ''' 
    This function creates spaces between word entities
    for simplification of further split() routine
    '''
    n1, n2 = a.shape
    for i in range(n1):
        for j in range(n2):
            a[i][j] = preprocess(a[i][j], tokenizer)

In [15]:
tokenize_array(X_train, tokenizer)
tokenize_array(X_test, tokenizer)

In [16]:
assert X_train[5][0] == '1 - к квартира , 33 м² , 4 / 5 эт .'
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'
assert X_test[2][0] == 'фара правая toyota rav 4 галоген 2015 - 19'
assert X_test[2][1] == 'фара правая для toyota rav4 2015 / оригинальный номер : 8113042650 / тойота рав4 тоета рав 4 / производитель : toyota / состояние : отличное без дефектов ! / комментарий : после 2015 не ксенон галоген + диод / пожалуйста , уточняйте соответствие вашего заказа изображенному на фото . / звоните уточняйте по наличию предоставляется время на проверку детали / отправляем в регионы рф транспортными компаниями / . / всегда включен вайбер вацап по вопросам !/ дополнительное фото по запросу'

### BOW (3 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в `train data`
 - Для каждого примера из `train` посчитать, сколько раз каждое слово из словаря в нём встречается


 В `sklearn` есть `CountVectorizer`, но в этом задании его использовать нельзя.

__Задание:__ создайте словарь, где каждому токену соответствует количество раз, которое оно встретилось в `X_train`.

In [16]:
def dict_subrout(tmp: list, d: dict) -> dict:
    '''
    Subroutine function to count occurences
    '''
    for item in tmp:
        if item in d:
            d[item] += 1
        else:
            d[item] = 1
    return d

In [17]:
def create_freq_dict(a: np.array) -> dict:
    '''
    Function to create
    key-value dictionary for BOW model
    key: word, value: occurence count
    '''
    tokens_cnt = {}
    n1, n2 = a.shape
    for i in range(n1):
        for j in range(n2):
            tmp_arr = a[i][j].split()
            tokens_cnt = dict_subrout(tmp_arr, tokens_cnt)
    return tokens_cnt

In [54]:
tokens_cnt = create_freq_dict(X_train)

In [55]:
assert tokens_cnt['сапоги'] == 454

__Задание:__ выведите 10 самых частотных и 10 самых редких токенов.

In [20]:
tokens_cnt_sorted = np.array(sorted(tokens_cnt.items(), key=lambda x: x[1], reverse=True))
tokens_cnt_sorted

array([['/', '85802'],
       [',', '79117'],
       ['.', '65624'],
       ...,
       ['дооснастить', '1'],
       ['хлебозаводская', '1'],
       ['фрионом', '1']], dtype='<U246')

The most frequent words are:

In [21]:
for i in range(10):
    print(tokens_cnt_sorted[i][0])

/
,
.
-
в
и
на
./
:
с


The most rare words are:

In [22]:
for i in range(10):
    print(tokens_cnt_sorted[len(tokens_cnt_sorted) - 1 - i][0])

фрионом
хлебозаводская
дооснастить
беспрецедентно
понравившейся
объективную
столиц
петровского
гремят
шуршат


__Задание:__ оставьте в словаре только топ-10000 самых частотных токенов, также создайте отдельный список из этих слов.

In [23]:
tokens_cnt_top = tokens_cnt_sorted[:10000] #list of tuples

In [24]:
def convert_to_dict(lst: np.array) -> dict:
    '''
    Function to convert np.array to dictionary
    in case it's needed in further model building
    '''
    tmp_dict = {}
    for tup in lst:
        tmp_dict[tup[0]] = tup[1] 
    return tmp_dict

In [25]:
tokens_cnt = convert_to_dict(tokens_cnt_top)
tokens_list = np.array([elem[0] for elem in tokens_cnt_top])
tokens_list_all = np.array([elem[0] for elem in tokens_cnt_sorted])

__Задание:__ реализуйте функцию, которая переводит текст в вектор из чисел. То есть каждому токену из списка токенов сопоставляется количество раз, которое он встретился в тексте.

In [26]:
def text_to_bow(text: str, tokens_list: list) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указано количество его употреблений в предложении
    input: строка, список токенов
    output: вектор той же размерности, что и список токенов
    """
    bow = []
    s_prep = preprocess(text, tokenizer)
    tokens = s_prep.split()
    for item in tokens_list:
        bow.append(tokens.count(item))
    return np.array(bow)

In [27]:
example_text = text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml", tokens_list)
np.nonzero(example_text)[0]

array([   1,    4,   12,  565,  866, 1601, 2539, 4063])

In [28]:
example_text = text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml", tokens_list)

assert np.allclose(example_text.mean(), 0.0008)

Код почти проходит assert и возвращаются довольно близкие индексы

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и каждому тексту из `'description'` сопоставляет вектор.

In [29]:
def descr_to_bow(items: np.array, tokens_list: list) -> np.array:
    """ Для каждого описания товара возвращает вектор его bow """
    bow_output = []
    for i in range(items.shape[0]):
        bow_output.append(text_to_bow(items[i][1], tokens_list))
    return np.array(bow_output)

In [30]:
X_train_bow = descr_to_bow(X_train, tokens_list)
X_test_bow = descr_to_bow(X_test, tokens_list)

In [58]:
X_train

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ...,
       ['lada kalina , 2008',
        'всю информацию о кредитовании можно получить у сотрудников петровского автоцентра ./ / об автомобиле :/ - автомобиль приобретался у оф

In [60]:
print(np.linalg.norm(X_test_bow[0]))
print(np.linalg.norm(X_train_bow[-2]))

17.08800749063506
1.4142135623730951


In [32]:
assert X_train_bow.shape == (21000, 10000), X_test_bow.shape == (9000, 10000)
assert 0.005 < X_train_bow.mean() < 0.006
assert 0.005 < X_test_bow.mean() < 0.006

### Логистическая регрессия и SVC (0.5 балла)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVM с линейным ядром (`sklearn.svm.LinearSVC` или `sklearn.svm.SVC(kernel='linear')`) с базовыми параметрами. При необходимости можете увеличить максимальное число итераций. В качестве `random_state` возьмите 13.

_Подсказка: для того, чтобы было проще обучать, можно использовать [разреженные матрицы](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D0%B7%D1%80%D0%B5%D0%B6%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D1%86%D0%B0) - многие модели из `sklearn` умеют с ними работать. Соответствующий модуль из `scipy`: [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html). Нетрудно заметить, что в полученных BOW-матрицах очень много нулей. Если хранить в памяти только ненулевые элементы, можно сильно оптимизировать вычисления. Можете в этом убедиться:_

In [84]:
from scipy.sparse import csr_matrix, issparse

In [34]:
X_train_bow_csr = compression(X_train_bow)

[[ 0  0  0 ...  0  0  0]
 [ 0  2  1 ...  0  0  0]
 [ 4  5  3 ...  0  0  0]
 ...
 [42 14  4 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 1  0  4 ...  0  0  0]] array in memory (raw): 1680.000 Mb
[[ 0  0  0 ...  0  0  0]
 [ 0  2  1 ...  0  0  0]
 [ 4  5  3 ...  0  0  0]
 ...
 [42 14  4 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 1  0  4 ...  0  0  0]] array in memory (compressed): 8.606 Mb


In [35]:
#y_train check if sparse
issparse(y_train)

False

In [36]:
X_test_bow_csr = compression(X_test_bow)

[[10  1  6 ...  0  0  0]
 [ 0  1  1 ...  0  0  0]
 [10  1  2 ...  0  0  0]
 ...
 [10  1  3 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  9  9 ...  0  0  0]] array in memory (raw): 720.000 Mb
[[10  1  6 ...  0  0  0]
 [ 0  1  1 ...  0  0  0]
 [10  1  2 ...  0  0  0]
 ...
 [10  1  3 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  9  9 ...  0  0  0]] array in memory (compressed): 3.702 Mb


In [69]:
del X_train_bow
del X_test_bow

In [85]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [40]:
#logistic regression

logreg = LogisticRegression(random_state=13, max_iter=1000)
logreg.fit(X_train_bow_csr, y_train)
y_pred_lg = logreg.predict(X_test_bow_csr)

In [41]:
#accuracy for Logistic Regression
print(accuracy_score(y_test, y_pred_lg))
assert accuracy_score(y_test, y_pred_lg) > 0.695

0.6966666666666667


In [42]:
#LinearSVC

svc = LinearSVC(random_state=13, max_iter=10000)
svc.fit(X_train_bow_csr, y_train)
y_pred_svc = svc.predict(X_test_bow_csr)

/Users/ivan/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [43]:
#accuracy for LinearSVC
print(accuracy_score(y_test, y_pred_svc))
assert accuracy_score(y_test, y_pred_svc) > 0.68

0.684


### Модификация признаков (2 балла)

Прибавьте к соответствующим BOW-векторам BOW-вектора для `'title'` товара с некоторым весом. Изменится ли качество? Как вы можете это объяснить?

добавляю 'title' с единичным весом; корпус становится больше и модель учитывает теперь не только наличие слов в description, но в том числе и в title, то есть BOW модель стала богаче и содержит больше вхождений тех же самых слов; ну а это в свою очередь влияет на резльутаты классификации

In [40]:
def descr_to_bow_all(items: np.array, tokens_list: list) -> np.array:
    """ 
    Для каждого описания товара возвращает вектор его bow 
    (modified for title addition)
    """
    bow_output = []
    for i in range(items.shape[0]):
        bow_output.append(
            np.add(text_to_bow(items[i][0], tokens_list), text_to_bow(items[i][1], tokens_list))
    )
    return np.array(bow_output)

In [41]:
X_train_bow_all = descr_to_bow_all(X_train, tokens_list)
X_test_bow_all = descr_to_bow_all(X_test, tokens_list)

In [42]:
X_train_bow_all_csr = compression(X_train_bow_all)
X_test_bow_all_csr = compression(X_test_bow_all)

[[ 0  0  0 ...  0  0  0]
 [ 0  2  1 ...  0  0  0]
 [ 4  5  3 ...  0  0  0]
 ...
 [42 15  4 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 1  0  4 ...  0  0  0]] array in memory (raw): 1680.000 Mb
[[ 0  0  0 ...  0  0  0]
 [ 0  2  1 ...  0  0  0]
 [ 4  5  3 ...  0  0  0]
 ...
 [42 15  4 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 1  0  4 ...  0  0  0]] array in memory (compressed): 9.000 Mb
[[10  1  6 ...  0  0  0]
 [ 0  1  1 ...  0  0  0]
 [10  1  2 ...  0  0  0]
 ...
 [10  1  3 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0 10  9 ...  0  0  0]] array in memory (raw): 720.000 Mb
[[10  1  6 ...  0  0  0]
 [ 0  1  1 ...  0  0  0]
 [10  1  2 ...  0  0  0]
 ...
 [10  1  3 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0 10  9 ...  0  0  0]] array in memory (compressed): 3.868 Mb


In [43]:
#logistic regression ('Title' + 'Description')

logreg = LogisticRegression(random_state=13, max_iter=1000)
logreg.fit(X_train_bow_all_csr, y_train)
y_pred_lg_all = logreg.predict(X_test_bow_all_csr)
print(accuracy_score(y_test, y_pred_lg_all))

0.7787777777777778


In [44]:
#LinearSVC ('Title' + 'Description')

svc = LinearSVC(random_state=13, max_iter=1000)
svc.fit(X_train_bow_all_csr, y_train)
y_pred_svc_all = svc.predict(X_test_bow_all_csr)
print(accuracy_score(y_test, y_pred_svc_all))

0.7535555555555555


/Users/ivan/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Нормализуйте данные с помощью `MinMaxScaler` или `MinAbsScaler` перед обучением. Что станет с качеством и почему?

Все же `MaxAbsScaler`

In [168]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()

In [46]:
scaler.fit(X_train_bow_all_csr)
X_train_bow_all_csr_scaled = scaler.transform(X_train_bow_all_csr)
X_test_bow_all_csr_scaled = scaler.transform(X_test_bow_all_csr)

In [47]:
#logistic regression (MaxAbsScaled)

logreg = LogisticRegression(random_state=13, max_iter=1000)
logreg.fit(X_train_bow_all_csr_scaled, y_train)
y_pred_lg_all_scaled = logreg.predict(X_test_bow_all_csr_scaled)
print(accuracy_score(y_test, y_pred_lg_all_scaled))

0.7191111111111111


In [48]:
#LinearSVC (MaxAbsScaled)

svc = LinearSVC(random_state=13, max_iter=1000)
svc.fit(X_train_bow_all_csr_scaled, y_train)
y_pred_svc_all_scaled = svc.predict(X_test_bow_all_csr_scaled)
print(accuracy_score(y_test, y_pred_svc_all_scaled))

0.7626666666666667


Почему в данном случае использовать `StandardScaler` - не очень хорошая идея?

### Иная предобработка (1 балл)

**На выбор**:

- **либо** обучите модели, используя для предобработки токенизатор и лемматизатор `pymystem3.Mystem`.
- **либо** добавьте к предобработке стэмминг.

Сравните полученное сейчас качество с полученным ранее и сделайте вывод.

In [49]:
!pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to /private/var/folders/k6/bbc60q_546v72xm537by5d400000gn/T/pip-req-build-j0nzo0c7
  Running command git clone -q https://github.com/nlpub/pymystem3 /private/var/folders/k6/bbc60q_546v72xm537by5d400000gn/T/pip-req-build-j0nzo0c7
  Created wheel for pymystem3: filename=pymystem3-0.2.0-py3-none-any.whl size=9920 sha256=3188504ef464e4a0047cb4640e04fcd3eed48bdfb39e24c294c34cfa17686c20
  Stored in directory: /private/var/folders/k6/bbc60q_546v72xm537by5d400000gn/T/pip-ephem-wheel-cache-j3zhevql/wheels/72/92/6b/9521f1a0b0f77ec13f07bf5f87c50c531b1d8686cf8ad48b43
Successfully built pymystem3


In [96]:
from pymystem3 import Mystem
import re

In [97]:
mystem = Mystem()
X_train_mystem = np.copy(X_train) # create copies to not modify initial array
X_test_mystem = np.copy(X_test)

In [98]:
regex = r"[^\w\s]"    

In [99]:
def clear_string(s: str, regex: str, mystem) -> str:
    '''
    Function clears text from excess punctuation
    '''
    s = re.sub(regex, "", s)
    l = mystem.lemmatize(s)
    return ''.join(l).strip()

In [100]:
def lemmatize_freq_dict(a: np.array, regex: str, mystem) -> dict:
    '''
    Function creates lemmas of words and returns BOW-dictionary
    key: lemma, value: occurence count
    '''
    tokens_cnt = {}
    n1, n2 = a.shape
    for i in range(n1):
        for j in range(n2):
            tmp_arr = clear_string(a[i][j], regex, mystem).split()
            tokens_cnt = dict_subrout(tmp_arr, tokens_cnt)
    return tokens_cnt

In [55]:
freq_dict_mystem = lemmatize_freq_dict(X_train, regex, mystem)

In [56]:
tokens_list_mystem = [key for key in freq_dict_mystem]

In [103]:
def prepare_stem(a: np.array) -> np.array:
    '''
    Function transforms initial array to lemmatized
    '''
    n1, n2 = a.shape
    for i in range(n1):
        for j in range(n2):
            a[i][j] = clear_string(a[i][j], regex, mystem)

In [58]:
prepare_stem(X_train_mystem)
prepare_stem(X_test_mystem)

array([['винт сдвижной панель потолок бмв х5 е53',
        '3308  винт сдвижной панель потолок oem  54107199476  54 10 7 199 476    запчасть б  у в хороший состояние для bmw x5 e53   демонтировать с автомобиль из япония без пробег по рф  гарантия на весь запчасть 14 день  возможный замена  установка в автосервис партнер   примечание    для упрощение поиск запчасть сообщать мы внутренний код   3308   в наличие на склад  16 шт   проверять соответствие данный запчасть по оригинальный номер оем  ваш автомобиль по vin  более 1000 наименование запасной часть для bmw на наш склад в москва  отправлять по весь россия почта или транспортный компания'],
       ['сандалик totto для мальчик',
        'продавать сандалик totto для малыш  размер 20  по стелька 12 см'],
       ['фара правый toyota rav 4 галоген 2015  19',
        'фара правый для toyota rav4 2015  оригинальный номер  8113042650  тойота рав4 тоета рав 4  производитель  toyota  состояние  отличный без дефект   комментарий  после 2015 не

In [59]:
X_train_bow_mystem = descr_to_bow(X_train_mystem, tokens_list_mystem)
X_test_bow_mystem = descr_to_bow(X_test_mystem, tokens_list_mystem)

In [60]:
X_train_bow_mystem_csr = compression(X_train_bow_mystem)

[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]] array in memory (raw): 9370.200 Mb
[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]] array in memory (compressed): 8.837 Mb


In [61]:
X_test_bow_mystem_csr = compression(X_test_bow_mystem)

[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] array in memory (raw): 4015.800 Mb
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] array in memory (compressed): 3.705 Mb


In [62]:
#logistic regression (mystem)

logreg = LogisticRegression(random_state=13, max_iter=1000)
logreg.fit(X_train_bow_mystem_csr, y_train)
y_pred_lg_stem = logreg.predict(X_test_bow_mystem_csr)
print(accuracy_score(y_test, y_pred_lg_stem))

0.7252222222222222


In [63]:
#LinearSVC (mystem)

svc = LinearSVC(random_state=13, max_iter=10000)
svc.fit(X_train_bow_mystem_csr, y_train)
y_pred_svc_stem = svc.predict(X_test_bow_mystem_csr)
print(accuracy_score(y_test, y_pred_svc_stem))

0.7128888888888889


### TF-IDF (5 баллов)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В `sklearn` есть `TfidfVectorizer`, но в этом задании его использовать нельзя. Для простоты посчитайте общий tf-idf для `'title'` и `'description'` (то есть каждому объекту надо сопоставить вектор, где как документ будет рассматриваться конкатенация `'title'` и `'description'`).

__Задание:__ составьте словарь, где каждому слову из изначального списка будет соответствовать количество документов из `train`-части, где это слово встретилось.

In [64]:
def add_to_dict(tmp: set, df_dict: dict):
    '''
    Supply function to count df (document frequency)
    measure for each word in corpus
    '''
    for word in tmp:
        if word in df_dict:
            df_dict[word] += 1
        else:
            df_dict[word] = 1
    return df_dict

In [65]:
def create_df_dict(sample: np.array) -> dict:
    '''
    Function creates df dictionary
    '''
    df_dict = {}
    n1, n2 = sample.shape
    for i in range(n1):
        tmp = []
        for j in range(n2):
            tmp += sample[i][j].split()
        tmp = set(tmp)
        df_dict = add_to_dict(tmp, df_dict)
    return df_dict

In [66]:
word_document_cnt = create_df_dict(X_train)

In [67]:
assert word_document_cnt['размер'] == 2839

__Задание:__ реализуйте функцию, где тексту в соответствие ставится tf-idf вектор. Для вычисления IDF также необходимо число документов в `train`-части (параметр `n_documents_total`).

In [68]:
def text_to_tfidf(text: str, word_document_cnt: dict, tokens_list: list, n_documents_total: int) -> np.array:
    """
    Возвращает вектор, где для каждого слова из словаря
    указан tf-idf
    """
    tf_idf_vec = []
    tokens = text.split()
    for word in tokens_list:
        tf_tmp = tokens.count(word) / len(tokens) # tf
        idf_tmp = np.log(n_documents_total / word_document_cnt[word]) # idf
        tf_idf_tmp = tf_tmp * idf_tmp # tf-idf
        tf_idf_vec.append(tf_idf_tmp)
    return np.array(tf_idf_vec)

In [99]:
example_text = text_to_tfidf(
    'сдаётся уютный , тёплый гараж для стартапов в ml',
    word_document_cnt,
    tokens_list,
    n_documents_total=len(X_train)
)
assert np.allclose(np.linalg.norm(example_text), 1.4435668)

In [98]:
example_text = text_to_tfidf(
    'сдаётся уютный , тёплый гараж для стартапов в ml',
    word_document_cnt,
    tokens_list,
    n_documents_total=len(X_train)
)
assert 0.0003 < example_text.mean() < 0.0004

__Задание:__ а теперь реализуйте функцию, которая преобразует наш датасет и для каждого объекта сопоставляет вектор tf-idf. В качестве текстов используйте конкатенацию `'title'` и `'description'`.

In [70]:
def items_to_tfidf(items: np.array, word_document_cnt: dict, tokens_list: list, n_documents_total: int) -> np.array:
    """
    Для каждого товара возвращает его tf-idf вектор
    """
    tf_idf_items = []
    n1, n2 = items.shape
    for i in range(n1):
        tmp = []
        for j in range(n2):
            tmp += items[i][j].split()
        tmp_str = ' '.join(tmp)
        tf_idf_tmp = text_to_tfidf(tmp_str, word_document_cnt, tokens_list, n_documents_total)
        tf_idf_items.append(tf_idf_tmp)
    return np.array(tf_idf_items)

In [71]:
X_train_tfidf = items_to_tfidf(X_train, word_document_cnt, tokens_list, len(X_train))
X_test_tfidf = items_to_tfidf(X_test, word_document_cnt, tokens_list, len(X_train))

In [100]:
assert X_train_tfidf.shape == (21000, 10000), X_test_tfidf.shape == (9000, 10000)
assert 0.0002 < X_train_tfidf.mean() < 0.0004
assert 0.0002 < X_test_tfidf.mean() < 0.0004

__Задание:__ обучите логистическую регрессию и SVC, оцените качество (accuracy_score). Сделайте вывод.

In [73]:
X_train_tfidf_csr = compression(X_train_tfidf)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.02780371 0.01110134 ... 0.         0.         0.        ]
 [0.03384833 0.01882543 0.00901984 ... 0.         0.         0.        ]
 ...
 [0.08529778 0.01355431 0.00288635 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.03531999 0.         0.05019738 ... 0.         0.         0.        ]] array in memory (raw): 1680.000 Mb
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.02780371 0.01110134 ... 0.         0.         0.        ]
 [0.03384833 0.01882543 0.00901984 ... 0.         0.         0.        ]
 ...
 [0.08529778 0.01355431 0.00288635 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.03531999 0.         0.05019738 ... 0.         0.         0.        ]] array in memory (compressed): 9.000 Mb


In [74]:
X_test_tfidf_csr = compression(X_test_tfidf)

[[0.06396534 0.00284605 0.01363629 ... 0.         0.         0.        ]
 [0.         0.02008046 0.01603527 ... 0.         0.         0.        ]
 [0.09127638 0.00406122 0.00648618 ... 0.         0.         0.        ]
 ...
 [0.07385089 0.00328589 0.00787186 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.03227216 0.02319388 ... 0.         0.         0.        ]] array in memory (raw): 720.000 Mb
[[0.06396534 0.00284605 0.01363629 ... 0.         0.         0.        ]
 [0.         0.02008046 0.01603527 ... 0.         0.         0.        ]
 [0.09127638 0.00406122 0.00648618 ... 0.         0.         0.        ]
 ...
 [0.07385089 0.00328589 0.00787186 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.03227216 0.02319388 ... 0.         0.         0.        ]] array in memory (compressed): 3.868 Mb


In [130]:
del X_train_tfidf
del X_test_tfidf

TypeError: 'tuple' object is not callable

In [75]:
#Logistic regression (tf-idf)

logreg = LogisticRegression(random_state=13, max_iter=1000)
logreg.fit(X_train_tfidf_csr, y_train)
y_pred_lg = logreg.predict(X_test_tfidf_csr)

In [76]:
print(accuracy_score(y_test, y_pred_lg))
assert accuracy_score(y_test, y_pred_lg) > 0.675

0.6795555555555556


In [77]:
#LinearSVC (tf-idf)

svc = LinearSVC(random_state=13, max_iter=1000)
svc.fit(X_train_tfidf_csr, y_train)
y_pred_svc = svc.predict(X_test_tfidf_csr)

In [78]:
print(accuracy_score(y_test, y_pred_svc))
assert accuracy_score(y_test, y_pred_svc) > 0.79

0.7942222222222223


### Word Vectors (4 балла)

Давайте попробуем другой подход -- каждому слову сопоставим какое-то векторное представление (эмбеддинг) - но достаточно маленькой размерности. Таким образом мы сильно уменьшим количество параметров в модели.

Почитать про это подробнее можно тут:

- https://habr.com/ru/company/ods/blog/329410/

Вектора мы возьмём уже готовые (обученные на текстах из интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [150]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
# если не работает (возможно, у вас windows) - можете скачать файл по соответствующей ссылке

--2020-04-14 22:04:34--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-14 22:04:34--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc50eea3dc4101610521716931a8.dl.dropboxusercontent.com/cd/0/inline/A12E-ehkrSTpnC4u4y03whOPrmC9fcDwvF_zSVvbls4L73negGxLCph5WBCXBfiQMW6ww7d3d6WE8smXjyH0IHsQnMxD_9kR1Bgta1Cem7OZyA/file# [following]
--2020-04-14 22:04:35--  https://uc50eea3dc4101610521716931a8.dl.dropboxusercontent.com/cd/0/inline/A12E-ehkrSTpnC4u4y03whOPrmC9fcDwvF_zSVvbls4L73negGxLCph5WBCXBfiQMW6ww7d3d6WE8smXjyH0IHsQnMxD_9kR1Bgta1Cem7OZyA/file
Resolving uc50eea3dc4101610521716931a8.dl.dropboxuser

In [152]:
!tar -xzf ru.tar.gz
# распаковка файла - опять же, если не работает, распакуйте вручную

ru.bin: (Empty error message)
tar: Error exit delayed from previous errors.


In [17]:
!pip install gensim

In [55]:
import gensim
from gensim.models.wrappers import FastText

embedding_model = FastText.load_fasttext_format('ru.bin')

In [167]:
# как мы видим, каждому слову данная модель сопоставляет вектор размерности 300

print(embedding_model['привет'].shape)
print(embedding_model['привет'])

(300,)
[ 0.02916384  0.02167605  0.05127367 -0.00971958  0.0465235  -0.03945766
  0.02737866  0.00638128 -0.03774629 -0.04257201 -0.00995653  0.02291315
 -0.02301722  0.06697998 -0.03674482 -0.02403202 -0.05404469  0.01372932
  0.00926399 -0.0013149   0.11941359 -0.022448    0.04011497  0.06980549
  0.00407011 -0.09384539  0.03050164 -0.02578281 -0.03525181 -0.06603175
  0.04752798  0.05874675  0.01983666  0.06092105 -0.00957561  0.08307806
 -0.01288903  0.04705157  0.02198839 -0.00649013 -0.0171444   0.03302203
  0.02124882 -0.01902875 -0.05235172  0.03458685 -0.01409259 -0.07477519
  0.01916078  0.02985001  0.0086322   0.03051201  0.02831862  0.04549561
  0.00761138 -0.05459622  0.09056009 -0.08807947 -0.05420396 -0.04793203
 -0.05672329 -0.03025264 -0.03024072 -0.05890108 -0.03137474  0.03292617
  0.05440779 -0.04548327 -0.07266086 -0.09327219  0.07247883  0.0111061
  0.01824225 -0.10570452  0.05110046 -0.04659343 -0.03277056 -0.00803401
 -0.03978698  0.00826598 -0.01074128  0.01843

__Задание:__ реализуйте функцию, выдающую эмбеддинг для предложения - как сумму эмбеддингов токенов.

In [66]:
def sentence_embedding(sentence: str, embedding_model) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """
    sentence = re.sub(regex, '', sentence)
    tokens = sentence.split()
    s = np.zeros(300, )
    for token in tokens:
        if token in embedding_model.wv.vocab:
            s += embedding_model[token]
        else:
            s += np.zeros(300, )
    return np.array(s)

In [164]:
assert sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml', embedding_model).shape == (300,)
assert np.linalg.norm(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml', embedding_model)) == 2.6764746

AssertionError: 

In [165]:
np.linalg.norm(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml', embedding_model))

2.1345817719976847

__Задание:__ сделайте все то же, что в предыдущих пунктах -- реализуйте функцию, которая преобразует данные, а затем обучите логистическую регрессию и SVM, оцените качество. Сделайте вывод, что работает лучше - модель, основанная на TF-IDF, или модель, обученная на предобученных эмбеддингах?

In [76]:
def text_to_embedded(items: np.array, embedding_model) -> np.array:
    out = []
    n1, n2 = items.shape
    for i in range(n1):
        out.append(
            np.add(sentence_embedding(items[i][0], embedding_model), sentence_embedding(items[i][1], embedding_model))
        )
    return np.array(out)   

In [78]:
X_train_embedded = text_to_embedded(X_train, embedding_model)
X_test_embedded = text_to_embedded(X_test, embedding_model)

In [ ]:
logreg_embed = LogisticRegression(max_iter=1000)
logreg_embed.fit(X_train_embedded, y_train)
y_pred_embed = logreg_embed.predict(X_test_embedded)

In [87]:
print(accuracy_score(y_pred_embed, y_test))

0.579


In [91]:
svc_embed  = LinearSVC(max_iter=10000)
svc_embed.fit(X_train_embedded, y_train)
y_pred_embed_svc = svc_embed.predict(X_test_embedded)

/Users/ivan/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [92]:
print(accuracy_score(y_pred_embed_svc, y_test))

0.567


Accuracy больше в tfidf, поэтому tfidf более точно помогает решить задачу классификации, нежели предобученные эмбеддинги. Так получилось, что довольно много слов не находятся в эмбеддинг словаре, поэтому они входят в модель с нулевым весом и никак не учитывается. Tfidf же опирается исключительно на корпус (то есть title + description), поэтому довольно хорошо работает в рамках такой задачи.

### Что дальше? (8 баллов)

Для получения максимальной оценки вам нужно решить любые 2 пункта. Решение каждого пункта даст вам полтора балла:

1. Реализовать n-gram модели текстовой классификации (__2 балла__)

2. Поработать с другими эмбеддингами для слов (например `word2vec` или `GloVe`) (__2 балла__)

3. Применить другие способы токенизации (например, `pymorphy2`, `spaCy`) и в целом предобработки данных (стоп-слова, стэмминг, лемматизация) (__2 балла__)

4. Добиться качества > 0.82 на тестовых данных (попробуйте другие токенизаторы, предобработку текста, и любые другие идеи, которые вам придут в голову) (__2 балла__)

Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

### 1 N-gram model (было влом и получилась полная ерунда)

In [109]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /Users/ivan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [150]:
X_train_gram = np.copy(X_train)
X_test_gram = np.copy(X_test)

In [106]:
prepare_stem(X_train_gram)
prepare_stem(X_test_gram)

In [110]:
stopwords_rus = stopwords.words("russian")

In [137]:
def remove_stopwords(items: np.array) -> np.array:
    n1, n2 = items.shape
    for i in range(n1):
        for j in range(n2):
            tokens = items[i][j].split()
            items[i][j] = ' '.join([token for token in tokens if token not in stopwords_rus])

In [138]:
#remove_stopwords(X_train_gram)
#remove_stopwords(X_test_gram)

In [161]:
pipeline_svc = Pipeline([
    ("tfidf", TfidfVectorizer(preprocessor = lambda x: x)),
    ("svc", LinearSVC(random_state=13))
])

In [162]:
param_grid = {
    "tfidf__ngram_range": [(1, 1), (1, 2), (1, 3), (1, 4)],
    "tfidf__max_df": (0.25, 0.5, 0.75, 1.0),
    "svc__penalty": ['l1', 'l2'],
    "svc__C": [0.5, 1.0, 2.0, 3.0]
}

In [163]:
grid_search = GridSearchCV(pipeline_svc, param_grid, cv=3, n_jobs=2, verbose=3)
grid_search.fit(X_train_gram, y_train)

Fitting 3 folds for each of 128 candidates, totalling 384 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    1.8s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:    7.1s
[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed:   16.0s
[Parallel(n_jobs=2)]: Done 384 out of 384 | elapsed:   21.6s finished


TypeError: cannot use a string pattern on a bytes-like object

In [154]:
grid_search.best_estimator_.steps

[('tfidf',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.float64'>, encoding='utf-8',
                  input='content', lowercase=True, max_df=0.25, max_features=None,
                  min_df=1, ngram_range=(1, 2), norm='l2',
                  preprocessor=<function <lambda> at 0x1a2055fdd0>,
                  smooth_idf=True, stop_words=None, strip_accents=None,
                  sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                  tokenizer=<function <lambda> at 0x1a2055fe60>, use_idf=True,
                  vocabulary=None)),
 ('svc',
  LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
            intercept_scaling=1, loss='squared_hinge', max_iter=1000,
            multi_class='ovr', penalty='l2', random_state=13, tol=0.0001,
            verbose=0))]

In [155]:
y_pred_gram = grid_search.predict(X_test_gram)

In [156]:
print("{}".format(accuracy_score(y_pred_gram, y_test)))

0.3501111111111111


### 4

Попробуем добиться улучшения качества на тестовых данных (используя лемматизатор mystem и tf-idf)

In [82]:
word_doc_cnt_mystem = create_df_dict(X_train_mystem)

In [84]:
X_train_tfidf_mystem = items_to_tfidf(X_train_mystem, word_doc_cnt_mystem, tokens_list_mystem, len(X_train_mystem))
X_test_tfidf_mystem = items_to_tfidf(X_test_mystem, word_doc_cnt_mystem, tokens_list_mystem, len(X_train_mystem))

In [85]:
X_train_tfidf_mystem_csr = compression(X_train_tfidf_mystem)

[[1.07146276 0.99589254 0.43431782 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03579956 0.03579956 0.        ]
 [0.         0.         0.         ... 0.         0.         2.48806943]
 [0.         0.         0.11581809 ... 0.         0.         0.        ]] array in memory (raw): 9370.200 Mb
[[1.07146276 0.99589254 0.43431782 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03579956 0.03579956 0.        ]
 [0.         0.         0.         ... 0.         0.         2.48806943]
 [0.         0.         0.11581809 ... 0.         0.         0.        ]] array in memory (compressed): 9.253 Mb


In [86]:
X_test_tfidf_mystem_csr = compression(X_test_tfidf_mystem)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.11581809 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] array in memory (raw): 4015.800 Mb
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.11581809 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] array in memory (compressed): 3.868 Mb


In [87]:
#Logistic regression (mystem + tf-idf)

logreg = LogisticRegression(random_state=13, max_iter=1000)
logreg.fit(X_train_tfidf_mystem_csr, y_train)
y_pred_lg_tfidf_mystem = logreg.predict(X_test_tfidf_mystem_csr)
print(accuracy_score(y_test, y_pred_lg_tfidf_mystem))

0.7584444444444445


In [88]:
#LinearSVC (mystem + tf-idf)

svc = LinearSVC(random_state=13, max_iter=1000)
svc.fit(X_train_tfidf_mystem_csr, y_train)
y_pred_svc_tfidf_mystem = svc.predict(X_test_tfidf_mystem_csr)
print(accuracy_score(y_test, y_pred_svc_tfidf_mystem))

0.8257777777777778
